In [8]:
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
# from backtesting.test import SMA
from backtesting.lib import SignalStrategy, TrailingStrategy
from ta import momentum
import pandas as pd


def SMA(arr, n):
    return pd.Series(arr).rolling(n).mean()

def EMA(arr, n):
    return pd.Series(arr).ewm(span=n, adjust=False).mean()

def RSI_Indicator (arr, n):
    arr_series = pd.Series(arr)
    rsi_indicator =  momentum.RSIIndicator(arr_series, window=n)
    return rsi_indicator.rsi()
    

class RsiStrategy(Strategy):
    def init(self):
        # Load the price and RSI
        price = self.data.Close
        self.rsi = self.I(RSI_Indicator, price, 14)

    def next(self):
        # Buy when RSI crosses above 30
        if crossover(self.rsi, 30):
            self.buy()
        # Sell when RSI crosses below 70
        elif crossover(70, self.rsi):
            self.sell()
    
    

class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        # SMA = pd.Series(arr).rolling(n).mean()
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


symbol = 'AAPL'

# Define the start and end dates for the data
# start_date = '2022-01-11'
# end_date = '2023-01-01'

# set current date as end date
# end_date = dt.datetime.now().strftime('%Y-%m-%d')

# Download the data using yfinance
data = yf.download(symbol, start="2005-01-01", end="2023-01-01", interval='1d')



bt = Backtest(data, RsiStrategy, commission=.002,
              cash=1_000,
              exclusive_orders=True)
stats = bt.run()

bt.plot()
stats

[*********************100%***********************]  1 of 1 completed


/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2005-01-03 00:00:00
End                       2022-12-30 00:00:00
Duration                   6570 days 00:00:00
Exposure Time [%]                   52.306334
Equity Final [$]                    23.104687
Equity Peak [$]                    1230.85534
Return [%]                         -97.689531
Buy & Hold Return [%]            11396.407844
Return (Ann.) [%]                  -18.904989
Volatility (Ann.) [%]                19.82905
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -98.135265
Avg. Drawdown [%]                  -19.336962
Max. Drawdown Duration     6370 days 00:00:00
Avg. Drawdown Duration      933 days 00:00:00
# Trades                                   97
Win Rate [%]                        16.494845
Best Trade [%]                      29.843084
Worst Trade [%]                    -37.225647
Avg. Trade [%]                    